In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack, csr_matrix
import re


In [25]:
df=pd.read_csv("synthetic_final.csv")
df.sample(5)


,Task ID,Description,Assigned Person,Start Date,End Date,Status,Notes,Next Step,Task Type,Priority
286,TASK-287,Clean up legacy code in syndicate proactive mi...,Andrew Wheeler,2025-06-10,2025-06-27,In Progress,Now first foot among ability.,For east apply glass reality including.,Refactoring,Low
499,TASK-500,Validate functionality of orchestrate user-cen...,Matthew Cannon,2025-06-27,2025-07-17,Completed,Even of economy prevent magazine scene appear ...,Later party question cut water.,Testing,Low
213,TASK-214,Write test cases for whiteboard sticky schemas,Hunter Martin,2025-06-18,2025-06-26,Pending,Out evening affect meet.,Style list rock main.,Testing,Low
358,TASK-359,Implement feature for engineer efficient markets,Erica Martinez,2025-07-02,2025-07-21,Completed,Do movie consider child suffer.,Event foot give note about time whose side.,Feature Development,Low
496,TASK-497,Update user guide on unleash e-business e-serv...,Charles Shelton,2025-06-08,2025-06-27,Completed,Ago interview make safe religious former he.,Material PM cultural marriage discussion.,Documentation,Low


In [17]:
df['Description_Clean']=df['Description'].str.lower()


In [12]:
df['Description_Clean'] = df['Description_Clean'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df[['Description','Description_Clean']].head(3)

,Description,Description_Clean
0,Write test cases for repurpose best-of-breed s...,write test cases for repurpose bestofbreed sch...
1,Develop new unleash turn-key ROI,develop new unleash turnkey roi
2,Refactor module for repurpose leading-edge act...,refactor module for repurpose leadingedge acti...


In [ ]:
tfidf=TfidfVectorizer(max_features=500)
x_tfidf=tfidf.fit_transform(df['Description_Clean'])



  (0, 244)	0.28250700159592634
  (0, 214)	0.3485290320188442
  (0, 27)	0.3485290320188442
  (0, 86)	0.14751832788668548
  (0, 178)	0.3762233322396792
  (0, 18)	0.41907961710274233
  (0, 149)	0.2030054970766251
  (0, 22)	0.41907961710274233
  (0, 192)	0.3485290320188442
  (1, 54)	0.3985263147166764
  (1, 146)	0.33635426348008296
  (1, 224)	0.468229721293829
  (1, 222)	0.4205776872107403
  (1, 121)	0.4205776872107403
  (1, 189)	0.39374145331060645
  (2, 86)	0.16012303494161445
  (2, 178)	0.40836974386219316
  (2, 174)	0.3352436263260356
  (2, 141)	0.3352436263260356
  (2, 124)	0.4241050221094404
  (2, 62)	0.34163952148338067
  (2, 3)	0.37830910349653885
  (2, 119)	0.37830910349653885
  (3, 182)	0.34335899827918126
  (3, 218)	0.2756395898310634
  :	:
  (496, 226)	0.336382358743038
  (496, 227)	0.36022525293771873
  (496, 97)	0.39251034422423975
  (497, 86)	0.17331686432074067
  (497, 7)	0.3990907389348263
  (497, 154)	0.3990907389348263
  (497, 148)	0.4211733423118193
  (497, 13)	0.544647

In [32]:
df['Priority_code']=df['Priority'].map({'Low':0,'Medium':1,'High':2})

df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
df['Duration'] = (df['End Date'] - df['Start Date']).dt.days


X_numeric=df[['Priority_code','Duration']].fillna(0).values

In [33]:
df[['Priority_code','Priority']].head(10)


,Priority_code,Priority
0,0,Low
1,0,Low
2,0,Low
3,0,Low
4,0,Low
5,1,Medium
6,2,High
7,0,Low
8,1,Medium
9,0,Low


In [35]:

X_combined = hstack([x_tfidf, csr_matrix(X_numeric)])

In [36]:
le = LabelEncoder()
df['Task_Type_Code'] = le.fit_transform(df['Task Type'])
y = df['Task_Type_Code']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:
print("Final shapes X_train:", X_train.shape, "| y_train:", y_train.shape)
print("Task Type classes:", list(le.classes_))

✅ Final shapes → X_train: (400, 247) | y_train: (400,)
✅ Task Type classes: ['Bug Fix', 'Client Request', 'Code Review', 'Deployment', 'Documentation', 'Feature Development', 'Refactoring', 'Research', 'Testing']


In [39]:
df.to_csv("cleaned_tasks.csv", index=False)